# **Universidade Federal do Rio Grande do Norte**


---


###**Algoritmo e Estrutura de Dados II**
###**Departamento de Engenharia de Computação e Automação**

---

**Alunos:**

Ariadne Cecília Evangelista da Silva e Arthur Phelipe de Oliveira Queiroz



## **Comparação de Performance: Dijkstra Clássico vs Dijkstra com Min-Heap**




In [ ]:
!pip install codecarbon
print('\n[INFO] codecarbon Instalado com sucesso!')


[INFO] codecarbon Instalado com sucesso!


In [ ]:
!pip install pandas
print('\n[INFO] Pandas Instalado com sucesso!')


[INFO] Pandas Instalado com sucesso!


In [ ]:
!pip install pandas matplotlib scipy
print('\n[INFO] Pandas matplotlib scipy Instalado com sucesso!')


[INFO] Pandas matplotlib scipy Instalado com sucesso!


#### **Bibliotecas Usadas**

In [ ]:
import networkx as nx
import numpy as np
import random
import time
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import os
from codecarbon import EmissionsTracker

print('\n[INFO] Bibliotecas impoortadas com sucesso!')


[INFO] Bibliotecas impoortadas com sucesso!


#### **Preparação do ambiente**

Realizamos a fixação das sementes (SEED) para os geradores de números aleatórios (random e numpy). Este passo é fundamental para garantir a reprodutibilidade do experimento: ao usar sempre a mesma semente, asseguramos que a geração dos grafos e a seleção dos nós de partida aleatórios sejam idênticas em todas as execuções do notebook. Isso permite uma comparação e análise mais consistentes dos resultados.

In [ ]:
# Garantir reprodutibilidade
SEED = 30
random.seed(SEED)
np.random.seed(SEED)

Conforme definido nos parâmetros do experimento, incluímos grafos com tamanhos até 50.000 nós, com 15 repetições por tamanho. A execução completa destes testes levou aproximadamente 3h40min. Optamos por limitar o tamanho máximo a 50.000 nós, excluindo o tamanho de 100.000 sugerido na proposta, devido ao tempo estimado de execução consideravelmente maior (aproximadamente o dobro), o que poderia impactar a viabilidade de execução no ambiente do Google Colab, especialmente considerando os limites de tempo de sessão, além disso era necessário uma quantidade de memória maior.

In [ ]:
# Define os parâmetros do experimento
GRAPH_SIZES = [100, 500, 1_000, 5_000, 10_000, 50_000, 100_000]
REPETITIONS = 15
NUM_SOURCES = 5  # 5 nós aleatórios

# Parâmetro de densidade para grafo GNP
# p=0.01 - cria um grafo esparso e conectado para N grande
PROBABILITY = 0.01

results_data = []

print("\n[INFO] Configuração Inicial concluída. Seeds fixadas e bibliotecas importadas.")


[INFO] Configuração Inicial concluída. Seeds fixadas e bibliotecas importadas.


#### **MinHeap**

In [ ]:
class MinHeap:
    def __init__(self, array):
        self.vertexMap = {idx: idx for idx in range(len(array))}
        self.heap = self.buildHeap(array)

    def isEmpty(self):
        return len(self.heap) == 0

    def buildHeap(self, array):
        firstParentIdx = (len(array) - 2) // 2
        for currentIdx in reversed(range(firstParentIdx + 1)):
            self.siftDown(currentIdx, len(array) - 1, array)
        return array

    def siftDown(self, currentIdx, endIdx, heap):
        childOneIdx = currentIdx * 2 + 1
        while childOneIdx <= endIdx:
            childTwoIdx = currentIdx * 2 + 2 if currentIdx * 2 + 2 <= endIdx else -1

            # Compara a distância (índice 1 do par [vértice, distância])
            if childTwoIdx != -1 and heap[childTwoIdx][1] < heap[childOneIdx][1]:
                idxToSwap = childTwoIdx
            else:
                idxToSwap = childOneIdx

            if heap[idxToSwap][1] < heap[currentIdx][1]:
                self.swap(currentIdx, idxToSwap, heap)
                currentIdx = idxToSwap
                childOneIdx = currentIdx * 2 + 1
            else:
                return

    def siftUp(self, currentIdx, heap):
        parentIdx = (currentIdx - 1) // 2
        # Compara a distância (índice 1)
        while currentIdx > 0 and heap[currentIdx][1] < heap[parentIdx][1]:
            self.swap(currentIdx, parentIdx, heap)
            currentIdx = parentIdx
            parentIdx = (currentIdx - 1) // 2

    def remove(self):
        if self.isEmpty():
            return None
        self.swap(0, len(self.heap) - 1, self.heap)
        vertex, distance = self.heap.pop()
        self.vertexMap.pop(vertex)
        self.siftDown(0, len(self.heap) - 1, self.heap)
        return vertex, distance

    def swap(self, i, j, heap):
        self.vertexMap[heap[i][0]] = j
        self.vertexMap[heap[j][0]] = i
        heap[i], heap[j] = heap[j], heap[i]

    def update(self, vertex, value):
        idx = self.vertexMap[vertex]
        self.heap[idx] = (vertex, value)
        self.siftUp(idx, self.heap)

print("\n[INFO]Iniciando o experimento de comparação de desempenho...")

TRACK_CO2 = True # Ativa medição das emissões de CO2 durante a execução de cada algoritmo


[INFO]Iniciando o experimento de comparação de desempenho...


#### **Criação de Grafos**

In [ ]:
def generate_weighted_graph(N, p):

    # Geração do Grafo Aleatório (GNP)
    G = nx.gnp_random_graph(n=N, p=p)

    # Adiciona pesos inteiros positivos
    for u, v in list(G.edges()):
        G[u][v]['weight'] = random.randint(1, 10)
        # Garante que seja direcionado para o Dijkstra, duplicando a aresta com o mesmo peso
        G.add_edge(v, u, weight=G[u][v]['weight'])

    # Garante conectividade
    # Usamos o maior componente conectado se o grafo não for totalmente conectado
    if N > 1 and not nx.is_connected(G.to_undirected()):
        largest_cc = max(nx.connected_components(G.to_undirected()), key=len)
        G = G.subgraph(largest_cc).copy()

    # Atualiza N para o tamanho real do grafo conectado, se necessário
    N_final = len(G.nodes)

    # Converte para Lista de Adjacência
    # A lista deve ter o tamanho N_final, com o índice = nó
    adjacency_list = [[] for _ in range(N_final)]

    # Garante que os nós no grafo networkx sejam remapeados para 0..N-1 se necessário
    node_map = {node: i for i, node in enumerate(G.nodes())}

    for u, v, data in G.edges(data=True):
        u_mapped = node_map[u]
        v_mapped = node_map[v]
        weight = data['weight']
        adjacency_list[u_mapped].append([v_mapped, weight])

    return G, adjacency_list, N_final, node_map

#### **Implementação Clássica do Algoritmo de Dijkstra**

* `getVertexWithMinDistance(distances, visited)`: Esta é uma função auxiliar usada pelo algoritmo clássico. O trabalho dela é encontrar, entre todos os vértices que ainda não foram visitados, aquele que possui a menor distância conhecida a partir do nó de partida. Ela faz isso percorrendo a lista completa de distâncias e verificando quais vértices ainda não estão no conjunto.

* `dijkstrasAlgorithm_Classic(start, edges)`: É a implementação principal do algoritmo de Dijkstra Clássico. Ela calcula as distâncias mínimas do nó de partida para todos os outros nós no grafo representado pela lista de adjacência.

In [ ]:
# Função Auxiliar O(V) para o Dijkstra Clássico
def getVertexWithMinDistance(distances, visited):
    currentMinDistance = float("inf")
    vertex = -1

    # Busca linear em todos os vértices não visitados
    for vertexIdx, distance in enumerate(distances):
        if vertexIdx in visited:
            continue

        if distance <= currentMinDistance:
            vertex = vertexIdx
            currentMinDistance = distance

    return vertex, currentMinDistance

# Dijkstra Clássico: O(V^2 + E)
def dijkstrasAlgorithm_Classic(start, edges):
    numberOfVertices = len(edges)
    minDistances = [float("inf") for _ in range(numberOfVertices)]
    minDistances[start] = 0
    visited = set()

    while len(visited) != numberOfVertices:
        # Busca O(V)
        vertex, currentMinDistance = getVertexWithMinDistance(minDistances, visited)

        if currentMinDistance == float("inf"):
            break

        visited.add(vertex)

        for edge in edges[vertex]:
            destination, distanceToDestination = edge

            if destination in visited:
                continue

            newPathDistance = currentMinDistance + distanceToDestination
            currentDestinationDistance = minDistances[destination]

            if newPathDistance < currentDestinationDistance:
                minDistances[destination] = newPathDistance

    # Substitui 'inf' por -1
    return list(map(lambda x: -1 if x == float("inf") else x, minDistances))

#### **Implementação do Algoritmo de Dijkstra Otimizado que utiliza um Min-Heap**

* `dijkstrasAlgorithm_MinHeap(start, edges)`: calcula as distâncias mínimas do nó de partida (start) para todos os outros nós no grafo, assim como a versão clássica. No entanto, ela faz isso de uma maneira muito mais eficiente.

A principal diferença é que, em vez de fazer uma busca linear em todos os vértices não visitados para encontrar o de menor distância, esta versão usa a classe MinHeap que definimos anteriormente. O Min-Heap armazena os vértices e suas distâncias conhecidas e permite encontrar e remover o vértice com a menor distância de forma muito rápida (em tempo logarítmico, O(log V)).

In [ ]:
# Dijkstra Otimizado: O((V + E) * log(V))
def dijkstrasAlgorithm_MinHeap(start, edges):
    numberOfVertices = len(edges)
    minDistances = [float("inf") for _ in range(numberOfVertices)]
    minDistances[start] = 0

    # Inicializa o MinHeap
    minDistancesHeap = MinHeap([(idx, float("inf")) for idx in range(numberOfVertices)])
    minDistancesHeap.update(start, 0) # Atualiza a distância inicial no heap [25]

    while not minDistancesHeap.isEmpty():
        # Extrai o Mínimo (O(log V))
        vertex, currentMinDistance = minDistancesHeap.remove()

        if currentMinDistance == float("inf"):
            break

        for edge in edges[vertex]:
            destination, distanceToDestination = edge

            newPathDistance = currentMinDistance + distanceToDestination
            currentDestinationDistance = minDistances[destination]

            if newPathDistance < currentDestinationDistance:
                minDistances[destination] = newPathDistance
                # Atualiza o heap (O(log V))
                minDistancesHeap.update(destination, newPathDistance)

    # Converte 'inf' para -1
    return list(map(lambda x: -1 if x == float("inf") else x, minDistances))

## **Experimento**

Vamos executar o experimento para cada tamanho de grafo separadamente.

#### **1. Para N = 100**

In [ ]:
# Certifique-se de que results_data esteja inicializado se estiver executando pela primeira vez
if 'results_data' not in globals():
    results_data = []

# --- Executar para N = 100 ---
N = 100
print(f"\n--- Processando grafos com N={N} nós (Repetições: {REPETITIONS}) ---")

start_time_size = time.time() # Capitura do tempo

# A. Geração do Grafo e Lista de Adjacência
G_nx, adj_list, N_actual, node_map = generate_weighted_graph(N, PROBABILITY)

# Garante que as fontes sejam selecionadas a partir dos nós existentes (0 até N_actual - 1)
all_nodes = list(range(N_actual))

for rep in range(REPETITIONS):
    # B. Seleção de 5 Nós de Partida Aleatórios
    source_nodes = random.sample(all_nodes, k=NUM_SOURCES)

    for start_node in source_nodes:
        # 1. DIJKSTRA CLÁSSICO (O(V^2 + E))
        algo_name = 'Dijkstra Clássico (V^2)'
        tracker_classic = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
        if TRACK_CO2: tracker_classic.start()
        start_time = time.time()
        dists_classic = dijkstrasAlgorithm_Classic(start_node, adj_list)
        end_time = time.time()
        if TRACK_CO2: emissions_classic = tracker_classic.stop()
        else: emissions_classic = 0.0
        results_data.append({
            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_classic
        })

        # 2. DIJKSTRA COM MIN-HEAP (O((V+E) log V))
        algo_name = 'Dijkstra Min-Heap ((V+E)logV)'
        tracker_heap = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
        if TRACK_CO2: tracker_heap.start()
        start_time = time.time()
        dists_heap = dijkstrasAlgorithm_MinHeap(start_node, adj_list)
        end_time = time.time()
        if TRACK_CO2: emissions_heap = tracker_heap.stop()
        else: emissions_heap = 0.0
        results_data.append({
            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_heap
        })

        # 3. REFERÊNCIA NETWORKX
        algo_name = 'NetworkX (Referência)'
        start_node_original = [k for k, v in node_map.items() if v == start_node][0]
        tracker_nx = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
        if TRACK_CO2: tracker_nx.start()
        start_time = time.time()
        dists_nx_dict = nx.shortest_path_length(G_nx, source=start_node_original, weight='weight')
        dists_nx = [-1] * N_actual
        for node, dist in dists_nx_dict.items():
             dists_nx[node_map[node]] = dist
        end_time = time.time()
        if TRACK_CO2: emissions_nx = tracker_nx.stop()
        else: emissions_nx = 0.0
        results_data.append({
            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_nx
        })

end_time_size = time.time() # Tempo
print(f"\nColeta de dados para N={N} concluída. Total de resultados coletados até agora: {len(results_data)}")
print(f"Tempo total para N={N}: {end_time_size - start_time_size:.2f} segundos")

# Salvar os resultados parciais após cada tamanho de grafo
df_partial = pd.DataFrame(results_data)
output_partial_filename = '/content/drive/MyDrive/Trabalho AED2 - 3pts/dijkstra_partial_results.csv'
df_partial.to_csv(output_partial_filename, index=False)
print(f"Resultados parciais salvos em: {output_partial_filename}")


--- Processando grafos com N=100 nós (Repetições: 15) ---

Coleta de dados para N=100 concluída. Total de resultados coletados até agora: 289
Tempo total para N=100: 307.42 segundos
Resultados parciais salvos em: /content/drive/MyDrive/Trabalho AED2 - 3pts/dijkstra_partial_results.csv


#### **2. Para N = 500**

In [ ]:
# Certifique-se de que results_data esteja inicializado se estiver executando pela primeira vez
if 'results_data' not in globals():
    results_data = []

# --- Executar para N = 500 ---
N = 500
print(f"\n--- Processando grafos com N={N} nós (Repetições: {REPETITIONS}) ---")

start_time_size = time.time() # Capitura do tempo

# A. Geração do Grafo e Lista de Adjacência
G_nx, adj_list, N_actual, node_map = generate_weighted_graph(N, PROBABILITY)

# Garante que as fontes sejam selecionadas a partir dos nós existentes (0 até N_actual - 1)
all_nodes = list(range(N_actual))

for rep in range(REPETITIONS):
    # B. Seleção de 5 Nós de Partida Aleatórios
    source_nodes = random.sample(all_nodes, k=NUM_SOURCES)

    for start_node in source_nodes:
        # 1. DIJKSTRA CLÁSSICO (O(V^2 + E))
        algo_name = 'Dijkstra Clássico (V^2)'
        tracker_classic = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
        if TRACK_CO2: tracker_classic.start()
        start_time = time.time()
        dists_classic = dijkstrasAlgorithm_Classic(start_node, adj_list)
        end_time = time.time()
        if TRACK_CO2: emissions_classic = tracker_classic.stop()
        else: emissions_classic = 0.0
        results_data.append({
            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_classic
        })

        # 2. DIJKSTRA COM MIN-HEAP (O((V+E) log V))
        algo_name = 'Dijkstra Min-Heap ((V+E)logV)'
        tracker_heap = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
        if TRACK_CO2: tracker_heap.start()
        start_time = time.time()
        dists_heap = dijkstrasAlgorithm_MinHeap(start_node, adj_list)
        end_time = time.time()
        if TRACK_CO2: emissions_heap = tracker_heap.stop()
        else: emissions_heap = 0.0
        results_data.append({
            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_heap
        })

        # 3. REFERÊNCIA NETWORKX
        algo_name = 'NetworkX (Referência)'
        start_node_original = [k for k, v in node_map.items() if v == start_node][0]
        tracker_nx = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
        if TRACK_CO2: tracker_nx.start()
        start_time = time.time()
        dists_nx_dict = nx.shortest_path_length(G_nx, source=start_node_original, weight='weight')
        dists_nx = [-1] * N_actual
        for node, dist in dists_nx_dict.items():
             dists_nx[node_map[node]] = dist
        end_time = time.time()
        if TRACK_CO2: emissions_nx = tracker_nx.stop()
        else: emissions_nx = 0.0
        results_data.append({
            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_nx
        })

end_time_size = time.time() # Capture end time for this size
print(f"\nColeta de dados para N={N} concluída. Total de resultados coletados até agora: {len(results_data)}")
print(f"Tempo total para N={N}: {end_time_size - start_time_size:.2f} segundos")

# Salvar os resultados parciais após cada tamanho de grafo
df_partial = pd.DataFrame(results_data)
output_partial_filename = '/content/drive/MyDrive/Trabalho AED2 - 3pts/dijkstra_partial_results.csv'
df_partial.to_csv(output_partial_filename, index=False)
print(f"Resultados parciais salvos em: {output_partial_filename}")


--- Processando grafos com N=500 nós (Repetições: 15) ---

Coleta de dados para N=500 concluída. Total de resultados coletados até agora: 514
Tempo total para N=500: 307.72 segundos
Resultados parciais salvos em: /content/drive/MyDrive/Trabalho AED2 - 3pts/dijkstra_partial_results.csv


#### **3. Para N = 1000**

In [ ]:
# Certifique-se de que results_data esteja inicializado se estiver executando pela primeira vez
if 'results_data' not in globals():
    results_data = []

# --- Executar para N = 1000 ---
N = 1000
print(f"\n--- Processando grafos com N={N} nós (Repetições: {REPETITIONS}) ---")

start_time_size = time.time() # Capitura do tempo

# A. Geração do Grafo e Lista de Adjacência
G_nx, adj_list, N_actual, node_map = generate_weighted_graph(N, PROBABILITY)

# Garante que as fontes sejam selecionadas a partir dos nós existentes (0 até N_actual - 1)
all_nodes = list(range(N_actual))

for rep in range(REPETITIONS):
    # B. Seleção de 5 Nós de Partida Aleatórios
    source_nodes = random.sample(all_nodes, k=NUM_SOURCES)

    for start_node in source_nodes:
        # 1. DIJKSTRA CLÁSSICO (O(V^2 + E))
        algo_name = 'Dijkstra Clássico (V^2)'
        tracker_classic = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
        if TRACK_CO2: tracker_classic.start()
        start_time = time.time()
        dists_classic = dijkstrasAlgorithm_Classic(start_node, adj_list)
        end_time = time.time()
        if TRACK_CO2: emissions_classic = tracker_classic.stop()
        else: emissions_classic = 0.0
        results_data.append({
            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_classic
        })

        # 2. DIJKSTRA COM MIN-HEAP (O((V+E) log V))
        algo_name = 'Dijkstra Min-Heap ((V+E)logV)'
        tracker_heap = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
        if TRACK_CO2: tracker_heap.start()
        start_time = time.time()
        dists_heap = dijkstrasAlgorithm_MinHeap(start_node, adj_list)
        end_time = time.time()
        if TRACK_CO2: emissions_heap = tracker_heap.stop()
        else: emissions_heap = 0.0
        results_data.append({
            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_heap
        })

        # 3. REFERÊNCIA NETWORKX
        algo_name = 'NetworkX (Referência)'
        start_node_original = [k for k, v in node_map.items() if v == start_node][0]
        tracker_nx = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
        if TRACK_CO2: tracker_nx.start()
        start_time = time.time()
        dists_nx_dict = nx.shortest_path_length(G_nx, source=start_node_original, weight='weight')
        dists_nx = [-1] * N_actual
        for node, dist in dists_nx_dict.items():
             dists_nx[node_map[node]] = dist
        end_time = time.time()
        if TRACK_CO2: emissions_nx = tracker_nx.stop()
        else: emissions_nx = 0.0
        results_data.append({
            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_nx
        })

end_time_size = time.time() # Tempo
print(f"\nColeta de dados para N={N} concluída. Total de resultados coletados até agora: {len(results_data)}")
print(f"Tempo total para N={N}: {end_time_size - start_time_size:.2f} segundos")

# Salvar os resultados parciais após cada tamanho de grafo
df_partial = pd.DataFrame(results_data)
output_partial_filename = '/content/drive/MyDrive/Trabalho AED2 - 3pts/dijkstra_partial_results.csv'
df_partial.to_csv(output_partial_filename, index=False)
print(f"Resultados parciais salvos em: {output_partial_filename}")


--- Processando grafos com N=1000 nós (Repetições: 15) ---

Coleta de dados para N=1000 concluída. Total de resultados coletados até agora: 739
Tempo total para N=1000: 308.89 segundos
Resultados parciais salvos em: /content/drive/MyDrive/Trabalho AED2 - 3pts/dijkstra_partial_results.csv


#### **4. Para N = 5000**

In [ ]:
# Certifique-se de que results_data esteja inicializado se estiver executando pela primeira vez
if 'results_data' not in globals():
    results_data = []

# --- Executar para N = 5000 ---
N = 5000
print(f"\n--- Processando grafos com N={N} nós (Repetições: {REPETITIONS}) ---")

start_time_size = time.time() # Capture start time for this size

# A. Geração do Grafo e Lista de Adjacência
G_nx, adj_list, N_actual, node_map = generate_weighted_graph(N, PROBABILITY)

# Garante que as fontes sejam selecionadas a partir dos nós existentes (0 até N_actual - 1)
all_nodes = list(range(N_actual))

for rep in range(REPETITIONS):
    # B. Seleção de 5 Nós de Partida Aleatórios
    source_nodes = random.sample(all_nodes, k=NUM_SOURCES)

    for start_node in source_nodes:
        # 1. DIJKSTRA CLÁSSICO (O(V^2 + E))
        algo_name = 'Dijkstra Clássico (V^2)'
        tracker_classic = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
        if TRACK_CO2: tracker_classic.start()
        start_time = time.time()
        dists_classic = dijkstrasAlgorithm_Classic(start_node, adj_list)
        end_time = time.time()
        if TRACK_CO2: emissions_classic = tracker_classic.stop()
        else: emissions_classic = 0.0
        results_data.append({
            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_classic
        })

        # 2. DIJKSTRA COM MIN-HEAP (O((V+E) log V))
        algo_name = 'Dijkstra Min-Heap ((V+E)logV)'
        tracker_heap = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
        if TRACK_CO2: tracker_heap.start()
        start_time = time.time()
        dists_heap = dijkstrasAlgorithm_MinHeap(start_node, adj_list)
        end_time = time.time()
        if TRACK_CO2: emissions_heap = tracker_heap.stop()
        else: emissions_heap = 0.0
        results_data.append({
            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_heap
        })

        # 3. REFERÊNCIA NETWORKX
        algo_name = 'NetworkX (Referência)'
        start_node_original = [k for k, v in node_map.items() if v == start_node][0]
        tracker_nx = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
        if TRACK_CO2: tracker_nx.start()
        start_time = time.time()
        dists_nx_dict = nx.shortest_path_length(G_nx, source=start_node_original, weight='weight')
        dists_nx = [-1] * N_actual
        for node, dist in dists_nx_dict.items():
             dists_nx[node_map[node]] = dist
        end_time = time.time()
        if TRACK_CO2: emissions_nx = tracker_nx.stop()
        else: emissions_nx = 0.0
        results_data.append({
            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_nx
        })

end_time_size = time.time() # Tempo
print(f"\nColeta de dados para N={N} concluída. Total de resultados coletados até agora: {len(results_data)}")
print(f"Tempo total para N={N}: {end_time_size - start_time_size:.2f} segundos")

# Salvar os resultados parciais após cada tamanho de grafo
df_partial = pd.DataFrame(results_data)
output_partial_filename = '/content/drive/MyDrive/Trabalho AED2 - 3pts/dijkstra_partial_results.csv'
df_partial.to_csv(output_partial_filename, index=False)
print(f"Resultados parciais salvos em: {output_partial_filename}")


--- Processando grafos com N=5000 nós (Repetições: 15) ---

Coleta de dados para N=5000 concluída. Total de resultados coletados até agora: 964
Tempo total para N=5000: 400.10 segundos
Resultados parciais salvos em: /content/drive/MyDrive/Trabalho AED2 - 3pts/dijkstra_partial_results.csv


#### **5. Para N = 10000**

In [ ]:
# Certifique-se de que results_data esteja inicializado se estiver executando pela primeira vez
if 'results_data' not in globals():
    results_data = []

# --- Executar para N = 10000 ---
N = 10000
print(f"\n--- Processando grafos com N={N} nós (Repetições: {REPETITIONS}) ---")

start_time_size = time.time() # Capture start time for this size

# A. Geração do Grafo e Lista de Adjacência
G_nx, adj_list, N_actual, node_map = generate_weighted_graph(N, PROBABILITY)

# Garante que as fontes sejam selecionadas a partir dos nós existentes (0 até N_actual - 1)
all_nodes = list(range(N_actual))

for rep in range(REPETITIONS):
    # B. Seleção de 5 Nós de Partida Aleatórios
    source_nodes = random.sample(all_nodes, k=NUM_SOURCES)

    for start_node in source_nodes:
        # 1. DIJKSTRA CLÁSSICO (O(V^2 + E))
        algo_name = 'Dijkstra Clássico (V^2)'
        tracker_classic = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
        if TRACK_CO2: tracker_classic.start()
        start_time = time.time()
        dists_classic = dijkstrasAlgorithm_Classic(start_node, adj_list)
        end_time = time.time()
        if TRACK_CO2: emissions_classic = tracker_classic.stop()
        else: emissions_classic = 0.0
        results_data.append({
            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_classic
        })

        # 2. DIJKSTRA COM MIN-HEAP (O((V+E) log V))
        algo_name = 'Dijkstra Min-Heap ((V+E)logV)'
        tracker_heap = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
        if TRACK_CO2: tracker_heap.start()
        start_time = time.time()
        dists_heap = dijkstrasAlgorithm_MinHeap(start_node, adj_list)
        end_time = time.time()
        if TRACK_CO2: emissions_heap = tracker_heap.stop()
        else: emissions_heap = 0.0
        results_data.append({
            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_heap
        })

        # 3. REFERÊNCIA NETWORKX
        algo_name = 'NetworkX (Referência)'
        start_node_original = [k for k, v in node_map.items() if v == start_node][0]
        tracker_nx = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
        if TRACK_CO2: tracker_nx.start()
        start_time = time.time()
        dists_nx_dict = nx.shortest_path_length(G_nx, source=start_node_original, weight='weight')
        dists_nx = [-1] * N_actual
        for node, dist in dists_nx_dict.items():
             dists_nx[node_map[node]] = dist
        end_time = time.time()
        if TRACK_CO2: emissions_nx = tracker_nx.stop()
        else: emissions_nx = 0.0
        results_data.append({
            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_nx
        })

end_time_size = time.time() # Tempo
print(f"\nColeta de dados para N={N} concluída. Total de resultados coletados até agora: {len(results_data)}")
print(f"Tempo total para N={N}: {end_time_size - start_time_size:.2f} segundos")

# Salvar os resultados parciais após cada tamanho de grafo
df_partial = pd.DataFrame(results_data)
output_partial_filename = '/content/drive/MyDrive/Trabalho AED2 - 3pts/dijkstra_partial_results.csv'
df_partial.to_csv(output_partial_filename, index=False)
print(f"Resultados parciais salvos em: {output_partial_filename}")


--- Processando grafos com N=10000 nós (Repetições: 15) ---

Coleta de dados para N=10000 concluída. Total de resultados coletados até agora: 1189
Tempo total para N=10000: 727.00 segundos
Resultados parciais salvos em: /content/drive/MyDrive/Trabalho AED2 - 3pts/dijkstra_partial_results.csv


#### **6. Para N = 50000**

In [ ]:
# Certifique-se de que results_data esteja inicializado se estiver executando pela primeira vez
if 'results_data' not in globals():
    results_data = []

# --- Executar para N = 50000 ---
N = 50000
print(f"\n--- Processando grafos com N={N} nós (Repetições: {REPETITIONS}) ---")

start_time_size = time.time() # Capture start time for this size

# A. Geração do Grafo e Lista de Adjacência
G_nx, adj_list, N_actual, node_map = generate_weighted_graph(N, PROBABILITY)

# Garante que as fontes sejam selecionadas a partir dos nós existentes (0 até N_actual - 1)
all_nodes = list(range(N_actual))

for rep in range(REPETITIONS):
    # B. Seleção de 5 Nós de Partida Aleatórios
    source_nodes = random.sample(all_nodes, k=NUM_SOURCES)

    for start_node in source_nodes:
        # 1. DIJKSTRA CLÁSSICO (O(V^2 + E))
        algo_name = 'Dijkstra Clássico (V^2)'
        tracker_classic = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
        if TRACK_CO2: tracker_classic.start()
        start_time = time.time()
        dists_classic = dijkstrasAlgorithm_Classic(start_node, adj_list)
        end_time = time.time()
        if TRACK_CO2: emissions_classic = tracker_classic.stop()
        else: emissions_classic = 0.0
        results_data.append({
            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_classic
        })

        # 2. DIJKSTRA COM MIN-HEAP (O((V+E) log V))
        algo_name = 'Dijkstra Min-Heap ((V+E)logV)'
        tracker_heap = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
        if TRACK_CO2: tracker_heap.start()
        start_time = time.time()
        dists_heap = dijkstrasAlgorithm_MinHeap(start_node, adj_list)
        end_time = time.time()
        if TRACK_CO2: emissions_heap = tracker_heap.stop()
        else: emissions_heap = 0.0
        results_data.append({
            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_heap
        })

        # 3. REFERÊNCIA NETWORKX
        algo_name = 'NetworkX (Referência)'
        start_node_original = [k for k, v in node_map.items() if v == start_node][0]
        tracker_nx = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
        if TRACK_CO2: tracker_nx.start()
        start_time = time.time()
        dists_nx_dict = nx.shortest_path_length(G_nx, source=start_node_original, weight='weight')
        dists_nx = [-1] * N_actual
        for node, dist in dists_nx_dict.items():
             dists_nx[node_map[node]] = dist
        end_time = time.time()
        if TRACK_CO2: emissions_nx = tracker_nx.stop()
        else: emissions_nx = 0.0
        results_data.append({
            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_nx
        })

end_time_size = time.time() # Tempo
print(f"\nColeta de dados para N={N} concluída. Total de resultados coletados até agora: {len(results_data)}")
print(f"Tempo total para N={N}: {end_time_size - start_time_size:.2f} segundos")

# Salvar os resultados parciais após cada tamanho de grafo
df_partial = pd.DataFrame(results_data)
output_partial_filename = '/content/drive/MyDrive/Trabalho AED2 - 3pts/dijkstra_partial_results.csv'
df_partial.to_csv(output_partial_filename, index=False)
print(f"Resultados parciais salvos em: {output_partial_filename}")


--- Processando grafos com N=50000 nós (Repetições: 15) ---

Coleta de dados para N=50000 concluída. Total de resultados coletados até agora: 1414
Tempo total para N=50000: 11492.87 segundos
Resultados parciais salvos em: /content/drive/MyDrive/Trabalho AED2 - 3pts/dijkstra_partial_results.csv


#### **7. Para N = 100000**
***OBS:** Vai dar problema de execução devido ao tempo e o alto consumo de memória*

In [ ]:
# Certifique-se de que results_data esteja inicializado se estiver executando pela primeira vez
#if 'results_data' not in globals():
#    results_data = []

# --- Executar para N = 100000 ---
# Observe que este tamanho pode levar um tempo considerável para executar.
#N = 100000
#print(f"\n--- Processando grafos com N={N} nós (Repetições: {REPETITIONS}) ---")

#start_time_size = time.time() # Capture start time for this size

# A. Geração do Grafo e Lista de Adjacência
#G_nx, adj_list, N_actual, node_map = generate_weighted_graph(N, PROBABILITY)

# Garante que as fontes sejam selecionadas a partir dos nós existentes (0 até N_actual - 1)
#all_nodes = list(range(N_actual))

#for rep in range(REPETITIONS):
    # B. Seleção de 5 Nós de Partida Aleatórios
#    source_nodes = random.sample(all_nodes, k=NUM_SOURCES)

#    for start_node in source_nodes:
#        # 1. DIJKSTRA CLÁSSICO (O(V^2 + E))
#        algo_name = 'Dijkstra Clássico (V^2)'
#        tracker_classic = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
#        if TRACK_CO2: tracker_classic.start()
#        start_time = time.time()
#        dists_classic = dijkstrasAlgorithm_Classic(start_node, adj_list)
#        end_time = time.time()
#        if TRACK_CO2: emissions_classic = tracker_classic.stop()
#        else: emissions_classic = 0.0
#        results_data.append({
#            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
#            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_classic
#        })

        # 2. DIJKSTRA COM MIN-HEAP (O((V+E) log V))
#        algo_name = 'Dijkstra Min-Heap ((V+E)logV)'
#        tracker_heap = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
#        if TRACK_CO2: tracker_heap.start()
#        start_time = time.time()
#        dists_heap = dijkstrasAlgorithm_MinHeap(start_node, adj_list)
#        end_time = time.time()
#        if TRACK_CO2: emissions_heap = tracker_heap.stop()
#        else: emissions_heap = 0.0
#        results_data.append({
#            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
#            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_heap
#        })

        # 3. REFERÊNCIA NETWORKX
#        algo_name = 'NetworkX (Referência)'
#        start_node_original = [k for k, v in node_map.items() if v == start_node][0]
#        tracker_nx = EmissionsTracker(output_dir=".", save_to_file=False, log_level="error")
#        if TRACK_CO2: tracker_nx.start()
#        start_time = time.time()
#        dists_nx_dict = nx.shortest_path_length(G_nx, source=start_node_original, weight='weight')
#        dists_nx = [-1] * N_actual
#        for node, dist in dists_nx_dict.items():
#             dists_nx[node_map[node]] = dist
#        end_time = time.time()
#        if TRACK_CO2: emissions_nx = tracker_nx.stop()
#        else: emissions_nx = 0.0
#        results_data.append({
#            'Tamanho_N': N_actual, 'Algoritmo': algo_name, 'Repeticao': rep,
#            'No_Fonte': start_node, 'Tempo_s': end_time - start_time, 'CO2_Emissoes_kg': emissions_nx
#        })

#end_time_size = time.time() # Tempo
#print(f"\nColeta de dados para N={N} concluída. Total de resultados coletados até agora: {len(results_data)}")
#print(f"Tempo total para N={N}: {end_time_size - start_time_size:.2f} segundos")

# Salvar os resultados parciais após cada tamanho de grafo
#df_partial = pd.DataFrame(results_data)
#output_partial_filename = '/content/drive/MyDrive/Trabalho AED2 - 3pts/dijkstra_partial_results.csv'
#df_partial.to_csv(output_partial_filename, index=False)
#print(f"Resultados parciais salvos em: {output_partial_filename}")


--- Processando grafos com N=100000 nós (Repetições: 15) ---


## Tratamento do Arquivo `.CSV` gerado

O arquivo `dijkstra_partial_results.csv` é a base de dados completa e bruta com todos os resultados registrados de cada teste individual realizado para os diferentes tamanhos de grafo e algoritmos.

Esta seção foca no tratamento e carregamento inicial desses dados. A célula de código abaixo lê este arquivo CSV, exibindo o total de registros coletados e as primeiras linhas para inspeção. Com base neste DataFrame, podemos iniciar as primeiras análises estatísticas e obter impressões preliminares sobre o desempenho de cada algoritmo em diferentes tamanhos.

In [ ]:
import pandas as pd
import os

# Define o caminho para o arquivo CSV no Google Drive
output_filename = '/content/drive/MyDrive/Trabalho AED2 - 3pts/dijkstra_partial_results.csv'

# Verifica se o arquivo existe antes de tentar ler
if os.path.exists(output_filename):
    # Cria o DataFrame a partir dos dados salvos no arquivo CSV
    df_results = pd.read_csv(output_filename)
    # Modificado para corresponder ao formato solicitado pelo usuário
    print(f"Dados carregados com sucesso do arquivo: {os.path.basename(output_filename)}")
    print(f"Total de registros de execução: {len(df_results)}")

    # Exibe as primeiras linhas do DataFrame para inspeção
    print("\nPrimeiras linhas dos resultados:")
    print(df_results.head(10))

    # Informações estatísticas preliminares (para a próxima fase de análise)
    print("\nEstatísticas Preliminares (Tempo Médio por Tamanho/Algoritmo):")
    summary = df_results.groupby(['Tamanho_N', 'Algoritmo'])['Tempo_s'].agg(['mean', 'std'])
    print(summary)
else:
    print(f"Erro: O arquivo {output_filename} não foi encontrado.")

Dados carregados com sucesso do arquivo: dijkstra_partial_results.csv
Total de registros de execução: 1414

Primeiras linhas dos resultados:
   Tamanho_N                      Algoritmo  Repeticao  No_Fonte   Tempo_s  \
0         19        Dijkstra Clássico (V^2)          0        17  0.000044   
1         19  Dijkstra Min-Heap ((V+E)logV)          0        17  0.000108   
2         19          NetworkX (Referência)          0        17  0.001353   
3         19        Dijkstra Clássico (V^2)          0         4  0.000069   
4         19  Dijkstra Min-Heap ((V+E)logV)          0         4  0.000104   
5         19          NetworkX (Referência)          0         4  0.000174   
6         19        Dijkstra Clássico (V^2)          0         1  0.000076   
7         19  Dijkstra Min-Heap ((V+E)logV)          0         1  0.000102   
8         19          NetworkX (Referência)          0         1  0.000216   
9         19        Dijkstra Clássico (V^2)          0        10  0.000042   



## Análise e Resumo Estátistico

Nesse trecho, pegamos os dados brutos de todas as execuções, agrupamos por tamanho do grafo e algoritmo, calculamos as médias, desvios-padrão e a margem do intervalo de confiança para o tempo e as emissões de CO₂. O resultado é salvo em um arquivo CSV de resumo `dijkstra_analysis_summary.csv`

In [ ]:
import scipy.stats # Importar especificamente o módulo stats

# 1. Agregação de Médias e Desvios-Padrão
df_summary = df_results.groupby(['Tamanho_N', 'Algoritmo']).agg(
    mean_time=('Tempo_s', 'mean'),
    std_time=('Tempo_s', 'std'),
    mean_co2=('CO2_Emissoes_kg', 'mean'),
    std_co2=('CO2_Emissoes_kg', 'std'),
    count=('Tempo_s', 'count')
).reset_index()

# 2. Cálculo dos Intervalos de Confiança (95% de confiança)
# Usaremos o Intervalo de Confiança (IC) para a Média.
# IC = Média +/- (t_score * Desvio_Padrão / sqrt(N))
# Para N > 30, o t_score é próximo de 1.96.
# Como o projeto exige 15-20 repetições, usamos a distribuição t de Student.

confidence_level = 0.95

def calculate_ci_margin(row):
    """Calcula a margem de erro para um dado nível de confiança."""
    if row['count'] < 2:
        return 0.0, 0.0 # Retorna 0.0 para ambos se count < 2 para evitar erros

    # Graus de liberdade
    df = row['count'] - 1
    # Valor T para 95% de confiança
    t_score = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, df)

    # Margem de Erro
    margin_time = t_score * row['std_time'] / np.sqrt(row['count'])
    margin_co2 = t_score * row['std_co2'] / np.sqrt(row['count'])

    return margin_time, margin_co2

# Aplica o cálculo e cria novas colunas para a margem de erro
df_summary[['ci_margin_time', 'ci_margin_co2']] = df_summary.apply(
    lambda row: pd.Series(calculate_ci_margin(row)), axis=1
)

# 3. Salva a tabela de análise consolidada (Entregável CSV)
# Define o caminho para o arquivo CSV no Google Drive
analysis_filename = '/content/drive/MyDrive/Trabalho AED2 - 3pts/dijkstra_analysis_summary.csv'
df_summary.to_csv(analysis_filename, index=False)
print(f"\nTabela de Análise Estatística salva em: {analysis_filename}")

print("\nResumo Estatístico (Tempo e CO2 Médios):")
print(df_summary[['Tamanho_N', 'Algoritmo', 'mean_time', 'std_time', 'ci_margin_time']].head(9))


Tabela de Análise Estatística salva em: /content/drive/MyDrive/Trabalho AED2 - 3pts/dijkstra_analysis_summary.csv

Resumo Estatístico (Tempo e CO2 Médios):
   Tamanho_N                      Algoritmo  mean_time  std_time  \
0         17        Dijkstra Clássico (V^2)   0.000041  0.000010   
1         17  Dijkstra Min-Heap ((V+E)logV)   0.000065  0.000017   
2         17          NetworkX (Referência)   0.000141  0.000021   
3         19        Dijkstra Clássico (V^2)   0.000047  0.000012   
4         19  Dijkstra Min-Heap ((V+E)logV)   0.000072  0.000017   
5         19          NetworkX (Referência)   0.000259  0.000286   
6        497        Dijkstra Clássico (V^2)   0.001421  0.002051   
7        497  Dijkstra Min-Heap ((V+E)logV)   0.000795  0.000339   
8        497          NetworkX (Referência)   0.002546  0.001119   

   ci_margin_time  
0        0.000002  
1        0.000004  
2        0.000005  
3        0.000005  
4        0.000008  
5        0.000130  
6        0.000472  
7 

## Gráficos

Agora, vamos gerar os gráficos para visualizar a comparação de desempenho e emissões de CO₂ dos algoritmos.

#### **Visualização: Comparação de Tempo Médio de Execução por Tamanho do Grafo**

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Certifique-se de que a coluna Tamanho_N seja numérica para o Plotly
df_summary['Tamanho_N'] = pd.to_numeric(df_summary['Tamanho_N'])

# Gráfico Interativo de Tempo Médio de Execução vs. Tamanho do Grafo (Plotly)
fig_time_plotly = px.line(df_summary,
                          x='Tamanho_N',
                          y='mean_time',
                          color='Algoritmo',
                          log_x=True,
                          log_y=True,
                          markers=True,
                          # Adiciona barras de erro usando a margem do intervalo de confiança
                          error_y='ci_margin_time',
                          title='Comparação de Tempo Médio de Execução por Tamanho do Grafo')


fig_time_plotly.update_layout(
    xaxis_title='Tamanho do Grafo (N - escala logarítmica)',
    yaxis_title='Tempo Médio de Execução (s - escala logarítmica)',
    hovermode='x unified' # Melhora a interatividade ao passar o mouse
)

fig_time_plotly.show()

# Salvar o gráfico interativo como HTML (opcional)
output_plot_time_plotly = '/content/drive/MyDrive/Trabalho AED2 - 3pts/tempo_medio_comparacao_plotly.html'
fig_time_plotly.write_html(output_plot_time_plotly)
print(f"\nGráfico Interativo de Tempo Médio salvo em: {output_plot_time_plotly}")


Gráfico Interativo de Tempo Médio salvo em: /content/drive/MyDrive/Trabalho AED2 - 3pts/tempo_medio_comparacao_plotly.html


## **Interpretação do Gráfico de Tempo Médio por Tamanho do Grafo**

 O gráfico mostra quanto tempo cada "solucionador" (algoritmo) leva para resolver o problema conforme ele fica maior.

 * **Dijkstra Clássico:** É método menos eficiente. No gráfico, a linha dele sobe muito rápido. Isso significa que, quando o grafo fica só um pouco maior, o tempo que ele leva para terminar "explode" (aumenta muito, muito rápido). É por isso que ele não conseguiu rodar para o grafo gigante de 100.000 nós.

* **Dijkstra com Min-Heap:** Esta é uma versão melhorada, mais "inteligente". A linha dele também sobe conforme o grafo aumenta, mas de forma muito mais suave e controlada. Mesmo quando o grafo fica bem grande, o tempo que ele leva aumenta, mas não "explode" como o clássico. Ele lida muito melhor com problemas grandes.

* **NetworkX (Referência):** A ferramenta já pronta que usamos (NetworkX) usa um método parecido com o Dijkstra com Min-Heap, por isso a linha dele se parece muito com a do Min-Heap, mostrando que também é eficiente para grafos grandes.

As Barras Verticais são as barras de Erro, elas mostram o quanto os tempos de execução variaram para aquele algoritmo e tamanho de grafo específico ao longo das 15 repetições.

  *   **Barras Curtas:** Significam que os tempos de execução para aquele algoritmo e tamanho foram bastante consistentes entre as repetições. A média calculada é mais "confiável".
  *   **Barras Longas:** Indicam que houve uma variação maior nos tempos de execução entre as repetições. A média calculada tem uma incerteza maior associada a ela.
  
Observe que, especialmente para o Dijkstra Clássico em tamanhos de grafo maiores, as barras de erro podem se tornar bastante longas, refletindo a grande variabilidade e o comportamento menos previsível do algoritmo quando ele começa a ficar sobrecarregado. Para os algoritmos mais eficientes, as barras tendem a ser mais curtas.
De modo geral, o gráfico mostra que, para grafos pequenos, todos são rápidos. Mas conforme o grafo cresce, o Dijkstra Clássico fica extremamente lento (a linha dispara para cima), enquanto o Dijkstra com Min-Heap e o NetworkX continuam relativamente rápidos (as linhas sobem bem menos).

A escala do gráfico ("logarítmica") ajuda a gente a ver essa diferença enorme de crescimento.



## **Gráfico de Emissões Médias de CO₂ vs. Tamanho do Grafo**

In [9]:
# Gráfico Interativo de Emissões Médias de CO₂ vs. Tamanho do Grafo
fig_co2_plotly = px.line(df_summary,
                         x='Tamanho_N',
                         y='mean_co2',
                         color='Algoritmo',
                         log_x=True,
                         markers=True,
                         # Adiciona barras de erro usando a margem do intervalo de confiança
                         error_y='ci_margin_co2',
                         title='Comparação de Emissões Médias de CO₂ por Tamanho do Grafo')

# Adicionar barras de erro (opcional, similar ao gráfico de tempo)

fig_co2_plotly.update_layout(
    xaxis_title='Tamanho do Grafo (escala logarítmica)',
    yaxis_title='Emissões Médias de CO₂ (kg)',
    hovermode='x unified' # Melhora a interatividade ao passar o mouse
)

fig_co2_plotly.show()

# Salvar o gráfico interativo como HTML (opcional)
output_plot_co2_plotly = '/content/drive/MyDrive/Trabalho AED2 - 3pts/co2_medio_comparacao_plotly.html'
fig_co2_plotly.write_html(output_plot_co2_plotly)
print(f"\nGráfico Interativo de Emissões de CO₂ Médias salvo em: {output_plot_co2_plotly}")


Gráfico Interativo de Emissões de CO₂ Médias salvo em: /content/drive/MyDrive/Trabalho AED2 - 3pts/co2_medio_comparacao_plotly.html


## **Interpretação do Gráfico de Emissões Médias de CO₂ vs. Tamanho do Grafo**

Assim como o tempo de execução, as emissões de CO₂ (a "pegada de carbono") estão diretamente ligadas ao quanto o computador trabalhou. Se um algoritmo leva mais tempo e usa mais recursos, ele tende a gerar mais emissões.

* **Dijkstra Clássico:** A linha dele para as emissões de CO₂ também sobe muito rapidamente conforme o grafo cresce. Assim como no gráfico de tempo, isso mostra que o algoritmo clássico, por ser menos eficiente para grafos grandes, faz o computador trabalhar muito mais e, consequentemente, gasta mais energia e gera mais CO₂.

* **Dijkstra com Min-Heap:** A linha para as emissões de CO₂ deste algoritmo sobe de forma muito mais suave e controlada, assim como no gráfico de tempo. Isso indica que, por ser mais eficiente e rápido, ele exige menos do computador e tem uma pegada de carbono menor para grafos grandes.

* **NetworkX (Referência):** A linha de CO₂ do NetworkX também segue de perto a do Min-Heap, reforçando que sua eficiência se traduz em menor consumo de energia e menores emissões de CO₂.

Em resumo, o segundo gráfico confirma o que vimos no primeiro: algoritmos mais eficientes (Min-Heap e NetworkX) não só rodam mais rápido, mas também são mais "verdes" ou consomem menos energia (e, portanto, geram menos CO₂ proporcionalmente ao tempo) do que o algoritmo clássico para grafos grandes. A diferença na pegada de carbono se torna enorme conforme o tamanho do problema aumenta.

## **Conclusão**

Algoritmos mais eficientes são mais rápidos e têm uma pegada de carbono consideravelmente menor para problemas maiores.

O **Dijkstra Clássico** se torna extremamente lento e consome muito mais recursos (gerando mais CO₂) à medida que o tamanho do grafo aumenta, tornando-o inviável para grafos grandes.

O **Dijkstra com Min-Heap e a implementação do NetworkX** demonstram ser muito mais escaláveis. Eles lidam com grafos grandes de forma muito mais eficiente, com um aumento de tempo e emissões de CO₂ significativamente menor em comparação com a versão clássica.

Isso mostra que a escolha do algoritmo correto, considerando sua eficiência (complexidade de tempo), tem um impacto direto e proporcional tanto na performance (rapidez) quanto no consumo de recursos computacionais (e, consequentemente, nas emissões de CO₂). Para problemas do mundo real com grandes conjuntos de dados (grafos), algoritmos otimizados como o Dijkstra com Min-Heap são essenciais não apenas pela velocidade, mas também por serem mais "sustentáveis" computacionalmente.